In [49]:
import importlib
import model.trainer
import torch
import data.VCTK
importlib.reload(model.trainer)
importlib.reload(data.VCTK)

<module 'data.VCTK' from '/home/azureuser/dom/Project/data/VCTK.py'>

In [ ]:
cpu = torch.device("cpu")

In [ ]:
trainer = model.trainer.Trainer(
    device=torch.device("cuda"),
    checkpoint=5
)

In [50]:
dataset = model.trainer.get_dataset()

In [51]:
batches = dataset.batched(4)

In [4]:
results = trainer.train_step(dataset[0])
results = [v.to(cpu) for v in results]
losses, predicted = results[:3], results[3:]
model.trainer.Trainer.show_loss(0, 0, *losses)

/home/azureuser/dom/Project/data/VCTK.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168786/work/torch/csrc/utils/tensor_new.cpp:201.)
  return text, torch.tensor(clips), torch.tensor(spectros)


epoch=0 i=0 total loss=101.91929 spectro loss=0.24191 vocoder loss=101.67738


In [5]:
x = [1,2,3]
x += [4]
x

[1, 2, 3, 4]